In [19]:
import os
import sys
import json
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import pickle

parent_dir = "/".join(sys.path[0].split("/")[:-1])
sys.path.append(parent_dir)

data_path = "../data/Cricket-Semantic-Segmentation"
annotations_path = "../data/Cricket-Semantic-Segmentation/annotations"


In [2]:
classes = {}
with open(f"{data_path}/classes/classes.json") as handler:
    classes = json.load(handler)
    classes = list(map(lambda obj: {"name":obj["name"], "color": obj["color"]}, classes))

In [3]:
allowed_colors = set(map(lambda cls: cls["color"], classes))
segments = list(filter(lambda name: "__fuse" in name, os.listdir(f"{data_path}/images")))

In [4]:
# from src.data.analyze import findColors

# with tqdm(total=len(segments)) as pbar:
#     for seg in segments:
#         img = cv.imread(f"{data_path}/images/{seg}")
#         available_colors = findColors(img)
#         ok = len(allowed_colors-available_colors) == len(allowed_colors)-len(available_colors)
#         if not ok:
#             print(seg, available_colors)
#         pbar.update(1)

In [26]:
from src.data.analyze import img2ColorMat
from src.data.process import getBoundingBoxesFromSegmentation

if not os.path.exists(annotations_path):
    os.makedirs(annotations_path)

label_names = ["Batsmen", "Ball", "Wicket"]
labels = list(filter(lambda cls: cls["name"] in label_names, classes))
with tqdm(total=len(segments)) as pbar:
    for i, seg in enumerate(segments):
        img = cv.imread(f"{data_path}/images/{seg}")
        boxes = getBoundingBoxesFromSegmentation(img, labels)
        txt_cntnt = "\n".join(list(map(lambda line: " ".join(list(map(lambda num: str(np.round(num, 6)), line))), boxes)))
        img_f_name = seg[:-11]
        txt_f_name = os.path.splitext(img_f_name)[0]+".txt"
        with open(f"{annotations_path}/{txt_f_name}", "w") as handler:
            handler.write(txt_cntnt)
        
        pbar.update(1)


  0%|          | 0/298 [00:00<?, ?it/s]

In [25]:
print(labels)

[{'name': 'Batsmen', 'color': '#b07a53'}, {'name': 'Ball', 'color': '#5e8d38'}, {'name': 'Wicket', 'color': '#1e472a'}]
